In [1]:
import pandas as pd
import numpy as np
df_policy = pd.read_csv('policy_claim/policy_0702.csv')
df_claim = pd.read_csv('policy_claim/claim_0702.csv')
df_training = pd.read_csv('training-set.csv')
df_testing = pd.read_csv('testing-set.csv')

## 0. Read Files

### policy

In [2]:
print(df_policy.shape) # 41 items  (1747942, 41)
print('unique:',df_policy['Policy_Number'].nunique()) #351273 unique Policy Number
pd.set_option('display.max_columns', None)
df_policy.head()   

(1747942, 41)
unique: 351273


,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
0,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16G,2000000,0,10000000,0,1551,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,
1,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16P,0,0,300000,0,1163,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,
2,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16G,5000000,0,60000000,0,2031,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,
3,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16P,0,0,500000,0,1098,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,
4,1a1f5f582bf1ff23467352e8f2fd31acefddc654,Fcd3d3578e9ae55aa6140da432f69db2bd0c50fad,bd864927fc796b75c917ade554c0657d22b860bd,,3828f4a4c40ac91647919306d0de0d11043d89fe,de36049ef6a91e6dd1a031ab5501beca28cc3610,cca82bf061cfd670f460f9e4a1b9ca853378382b,2006,1341,10,f4c3b98bf06237d20aa5f6c73b0c5359019e1cf8,5.0,P,竊盜,05N,0,0,65000,10,149,39.9,86418116e42c87c4d555d363b14b11827f39b8c4,0.0,6,0.3,0.0,1,12/1989,2,2,9ffd1ae121c4f26fe7f0c45ecdc85fa6ac245bf0,c1dfd96eea8cc2b62785275bca38ac261256e278,12/1989,0,0,0,0,0,0,0,


In [3]:
print(df_policy.columns)

Index(['Policy_Number', 'Insured's_ID', 'Prior_Policy_Number', 'Cancellation',
       'Vehicle_identifier', 'Vehicle_Make_and_Model1',
       'Vehicle_Make_and_Model2', 'Manafactured_Year_and_Month',
       'Engine_Displacement_(Cubic_Centimeter)', 'Imported_or_Domestic_Car',
       'Coding_of_Vehicle_Branding_&_Type', 'qpt', 'fpt',
       'Main_Insurance_Coverage_Group', 'Insurance_Coverage',
       'Insured_Amount1', 'Insured_Amount2', 'Insured_Amount3',
       'Coverage_Deductible_if_applied', 'Premium',
       'Replacement_cost_of_insured_vehicle', 'Distribution_Channel',
       'Multiple_Products_with_TmNewa_(Yes_or_No?)', 'lia_class', 'plia_acc',
       'pdmg_acc', 'fassured', 'ibirth', 'fsex', 'fmarriage', 'aassured_zip',
       'iply_area', 'dbirth', 'fequipment1', 'fequipment2', 'fequipment3',
       'fequipment4', 'fequipment5', 'fequipment6', 'fequipment9',
       'nequipment9'],
      dtype='object')


In [4]:
### examine Drivers
driver = df_policy[['ibirth' , 'dbirth']]
print(driver.shape[0])
driver.head()

1747942


,ibirth,dbirth
0,08/1984,08/1984
1,08/1984,08/1984
2,10/1975,10/1975
3,10/1975,10/1975
4,12/1989,12/1989


In [5]:
condition1 = driver['ibirth'].isnull()    # 18.2 %
condition2 = driver['dbirth'].isnull()    # 14.8 %

condition3 = driver['ibirth'].notnull()   
condition4 = driver['dbirth'].notnull()   


driver_0N= driver[condition3&condition4]  # both have values  (81.5%)
driver_1N= driver[condition1^condition2]  # missing one (3.9%)
driver_2N= driver[condition1|condition2]  # missing both (18.4%)

print(driver_0N.shape[0]/driver.shape[0])
print(driver_1N.shape[0]/driver.shape[0]) 
print(driver_2N.shape[0]/driver.shape[0])

0.8152427254451234
0.039585409584528546
0.18475727455487653


In [6]:
# check indiff. in 0N

indiff = driver_0N[driver_0N['ibirth']!=driver_0N['dbirth']]
print(indiff.shape[0])
indiff.sample(10)

75794


,ibirth,dbirth
215763,07/1975,07/1982
57342,08/1974,02/1974
454027,08/1946,11/1975
1292121,05/1967,11/1972
123922,02/1951,02/1971
852302,11/1970,05/1970
1252879,08/1974,03/1975
430576,08/1956,10/1984
813666,10/1952,05/1951
206416,05/1962,03/1987


In [7]:
### dbirth has some weird date
### check non-null data & sort
print(driver['dbirth'].notnull().sum())
import datetime
a = df_policy[driver['dbirth'].notnull()]['dbirth']
b = sorted(a, key=lambda x: datetime.datetime.strptime(x, '%m/%Y'), reverse = True)
b

1489010


['09/2658',
 '09/2658',
 '09/2658',
 '09/2658',
 '10/2622',
 '10/2622',
 '10/2622',
 '10/2622',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '10/2582',
 '10/2582',
 '10/2582',
 '10/2582',
 '10/2581',
 '10/2581',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '08/2569',
 '08/2569',
 '08/2569',
 '08/2569',
 '08/2569',
 '10/2521',
 '11/2512',
 '11/2512',
 '11/2512',
 '11/2512',
 '11/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '10/2472',
 '10/2472',
 '08/2471',
 '08/2471',
 '08/2471',
 '08/2471',
 '08/2471',
 '11/2464',
 '11/2464',
 '11/2464',
 '03/2464',
 '03/2464',
 '12/2453',
 '12/2453',
 '12/2453',
 '12/2453',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '02

In [8]:
replace_date = {
    '09/2658',
 '09/2658',
 '09/2658',
 '09/2658',
 '10/2622',
 '10/2622',
 '10/2622',
 '10/2622',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '02/2611',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '11/2602',
 '10/2582',
 '10/2582',
 '10/2582',
 '10/2582',
 '10/2581',
 '10/2581',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '04/2579',
 '08/2569',
 '08/2569',
 '08/2569',
 '08/2569',
 '08/2569',
 '10/2521',
 '11/2512',
 '11/2512',
 '11/2512',
 '11/2512',
 '11/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '01/2512',
 '10/2472',
 '10/2472',
 '08/2471',
 '08/2471',
 '08/2471',
 '08/2471',
 '08/2471',
 '11/2464',
 '11/2464',
 '11/2464',
 '03/2464',
 '03/2464',
 '12/2453',
 '12/2453',
 '12/2453',
 '12/2453',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '01/2442',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '07/2280',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152',
 '02/2152'}
len(replace_date)

19

In [9]:
replace_date_set = set(replace_date)
len(replace_date_set)

19

In [10]:
date_switch = df_policy.apply(lambda row: True if row['dbirth'] in replace_date_set else False, axis = 1)
df_policy_selected = df_policy[date_switch]
print(df_policy_selected.shape[0])
df_policy_selected.head()

92


,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
23221,26c9bba3cbb0b43a7309ca3844cb70b152d350bc,246fdc9ec76dd8a576402e1a284dc714fda7a9227,NaN,Y,9bae97ad226712d3596e1d3960f416a97f328bf0,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2010,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,竊盜,05N,0,0,1500000,10,2818,280.0,39005d92752d56310b3a357f42b57f304621689a,2.0,4,0.0,0.0,2,NaN,,,ecb7937db58ec9dea0c47db88463d85e81143032,356a192b7913b04c54574d18c28d46e6395428ab,08/2569,0,0,0,0,0,0,0,
23222,26c9bba3cbb0b43a7309ca3844cb70b152d350bc,246fdc9ec76dd8a576402e1a284dc714fda7a9227,NaN,Y,9bae97ad226712d3596e1d3960f416a97f328bf0,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2010,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,竊盜,09@,0,0,1500000,0,423,280.0,39005d92752d56310b3a357f42b57f304621689a,2.0,4,0.0,0.0,2,NaN,,,ecb7937db58ec9dea0c47db88463d85e81143032,356a192b7913b04c54574d18c28d46e6395428ab,08/2569,0,0,0,0,0,0,0,
23223,26c9bba3cbb0b43a7309ca3844cb70b152d350bc,246fdc9ec76dd8a576402e1a284dc714fda7a9227,NaN,Y,9bae97ad226712d3596e1d3960f416a97f328bf0,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2010,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,車責,16G,3000000,0,36000000,0,2178,280.0,39005d92752d56310b3a357f42b57f304621689a,2.0,4,0.0,0.0,2,NaN,,,ecb7937db58ec9dea0c47db88463d85e81143032,356a192b7913b04c54574d18c28d46e6395428ab,08/2569,0,0,0,0,0,0,0,
23224,26c9bba3cbb0b43a7309ca3844cb70b152d350bc,246fdc9ec76dd8a576402e1a284dc714fda7a9227,NaN,Y,9bae97ad226712d3596e1d3960f416a97f328bf0,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2010,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,車責,16P,0,0,600000,0,1680,280.0,39005d92752d56310b3a357f42b57f304621689a,2.0,4,0.0,0.0,2,NaN,,,ecb7937db58ec9dea0c47db88463d85e81143032,356a192b7913b04c54574d18c28d46e6395428ab,08/2569,0,0,0,0,0,0,0,
23225,26c9bba3cbb0b43a7309ca3844cb70b152d350bc,246fdc9ec76dd8a576402e1a284dc714fda7a9227,NaN,Y,9bae97ad226712d3596e1d3960f416a97f328bf0,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,d6c56ea1c2566966074bd939bcbf4c33d572988a,2010,3456,40,99805b8e11f38024b58281188d3065a45d04952a,5.0,P,車損,20K,10000,0,50000,0,21,280.0,39005d92752d56310b3a357f42b57f304621689a,2.0,4,0.0,0.0,2,NaN,,,ecb7937db58ec9dea0c47db88463d85e81143032,356a192b7913b04c54574d18c28d46e6395428ab,08/2569,0,0,0,0,0,0,0,


In [11]:
selected_idx = df_policy_selected.index.values
selected_idx

array([  23221,   23222,   23223,   23224,   23225,  162161,  162162,
        234348,  234349,  277667,  277668,  277669,  277670,  277671,
        277672,  283765,  283766,  283767,  283768,  283769,  283770,
        283771,  283772,  324704,  324705,  324706,  324707,  324708,
        324709,  324710,  324711,  324712,  418357,  418358,  418359,
        418360,  541847,  665698,  665699,  665700,  665701,  896783,
        896784,  896785,  896786,  896787,  896788,  896789,  896790,
        896791,  972345,  972346,  972347,  972348,  999213,  999214,
       1106167, 1106168, 1106169, 1106170, 1106171, 1420273, 1420274,
       1420275, 1420276, 1420277, 1457082, 1457083, 1457084, 1457085,
       1461117, 1461118, 1461119, 1521091, 1521092, 1521093, 1521094,
       1521095, 1521096, 1608188, 1608189, 1608190, 1608191, 1608192,
       1745912, 1745913, 1745914, 1745915, 1745916, 1745917, 1745918,
       1745919])

In [12]:
df_policy_selected_date = df_policy[['ibirth', 'dbirth','fassured']].loc[df_policy_selected.index]
# "1:國內自然人 2:國內法人 3:國外自然人 4:國外法人 5:自然人ID有誤 6:法人ID有誤"
df1 = df_policy_selected_date[df_policy_selected_date['fassured']==1]
df2 = df_policy_selected_date[df_policy_selected_date['fassured']==2]
print(df1.shape[0])
print(df2.shape[0])

60
32


In [13]:
df2.head() ## replace as NaN  (as ibirth)

,ibirth,dbirth,fassured
23221,NaN,08/2569,2
23222,NaN,08/2569,2
23223,NaN,08/2569,2
23224,NaN,08/2569,2
23225,NaN,08/2569,2


In [14]:
df1.head() ## replace as ibirth

,ibirth,dbirth,fassured
324704,04/1979,04/2579,1
324705,04/1979,04/2579,1
324706,04/1979,04/2579,1
324707,04/1979,04/2579,1
324708,04/1979,04/2579,1


In [15]:
df_policy['ID']=df_policy.index   # create ID column for ref.
df_policy.head()

,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9,ID
0,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16G,2000000,0,10000000,0,1551,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,0
1,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16P,0,0,300000,0,1163,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,1
2,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16G,5000000,0,60000000,0,2031,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,,2
3,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16P,0,0,500000,0,1098,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,,3
4,1a1f5f582bf1ff23467352e8f2fd31acefddc654,Fcd3d3578e9ae55aa6140da432f69db2bd0c50fad,bd864927fc796b75c917ade554c0657d22b860bd,,3828f4a4c40ac91647919306d0de0d11043d89fe,de36049ef6a91e6dd1a031ab5501beca28cc3610,cca82bf061cfd670f460f9e4a1b9ca853378382b,2006,1341,10,f4c3b98bf06237d20aa5f6c73b0c5359019e1cf8,5.0,P,竊盜,05N,0,0,65000,10,149,39.9,86418116e42c87c4d555d363b14b11827f39b8c4,0.0,6,0.3,0.0,1,12/1989,2,2,9ffd1ae121c4f26fe7f0c45ecdc85fa6ac245bf0,c1dfd96eea8cc2b62785275bca38ac261256e278,12/1989,0,0,0,0,0,0,0,,4


In [16]:
### replace weird date as ibirth
df_policy['dbirth_new'] = df_policy.apply(lambda row: row['ibirth'] if row['ID'] in selected_idx 
                                          else row['dbirth'], axis = 1)

In [17]:
df_policy[['ibirth', 'dbirth', 'fassured','dbirth_new']].loc[selected_idx].sample(10)

,ibirth,dbirth,fassured,dbirth_new
1608189,08/1967,08/2471,1,08/1967
1420273,02/1951,01/2512,1,02/1951
665698,10/1982,10/2622,1,10/1982
1461117,12/1958,11/2464,1,12/1958
277670,NaN,01/2442,2,NaN
324711,04/1979,04/2579,1,04/1979
896789,02/1952,02/2152,1,02/1952
324706,04/1979,04/2579,1,04/1979
896783,02/1952,02/2152,1,02/1952
1521095,02/1981,02/2611,1,02/1981


In [18]:
df_policy['dbirth'] = df_policy['dbirth_new']
df_policy = df_policy.drop(['dbirth_new','ID'], axis =1)
df_policy.head()

,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
0,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16G,2000000,0,10000000,0,1551,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,
1,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16P,0,0,300000,0,1163,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,
2,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16G,5000000,0,60000000,0,2031,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,
3,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16P,0,0,500000,0,1098,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,
4,1a1f5f582bf1ff23467352e8f2fd31acefddc654,Fcd3d3578e9ae55aa6140da432f69db2bd0c50fad,bd864927fc796b75c917ade554c0657d22b860bd,,3828f4a4c40ac91647919306d0de0d11043d89fe,de36049ef6a91e6dd1a031ab5501beca28cc3610,cca82bf061cfd670f460f9e4a1b9ca853378382b,2006,1341,10,f4c3b98bf06237d20aa5f6c73b0c5359019e1cf8,5.0,P,竊盜,05N,0,0,65000,10,149,39.9,86418116e42c87c4d555d363b14b11827f39b8c4,0.0,6,0.3,0.0,1,12/1989,2,2,9ffd1ae121c4f26fe7f0c45ecdc85fa6ac245bf0,c1dfd96eea8cc2b62785275bca38ac261256e278,12/1989,0,0,0,0,0,0,0,


In [19]:
print(df_policy.shape)

(1747942, 41)


### Claim

In [20]:
print(df_claim.shape)  # 20 items
print('unique:',df_claim['Policy_Number'].nunique()) #35895 unique Policy Number
pd.set_option('display.max_columns', None)  ## show all columns
df_claim.head()

(69613, 20)
unique: 35895


,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time
0,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,56705,0,0,00I,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,3000,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00
1,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,30000,700,0,16P,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00
2,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,1100,0,0,16G,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00
3,e33ba1121d6772dd84e83b266d66b1a7dea26d71,1,83ec7fa50d3f77f9739f0ce8d0139514e2a5f3e7,2,2,09/1975,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,151795,700,0,16G,0baf14fc451d6fef13ac82be90523fa7805df093,100.0,1,0,a17554a0d2b15a664c0e73900184544f19e70227,1.981,08:00
4,f26827172ff9528d11c66868cbf57f4e9c2d537c,1,bb616a58f59cd6d1b51c0343aa2aab2ff03a7b48,2,2,02/1971,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,0,0,0,16P,34209497dbd08b03d61c532acf939cd1f194bfd2,100.0,1,0,511a418e72591eb7e33f703f04c3fa16df6c90bd,1.981,08:00


In [21]:
print(df_claim.columns)

Index(['Claim_Number', 'Nature_of_the_claim', 'Policy_Number',
       'Driver's_Gender', 'Driver's_Relationship_with_Insured',
       'DOB_of_Driver', 'Marital_Status_of_Driver', 'Accident_Date',
       'Cause_of_Loss', 'Paid_Loss_Amount', 'paid_Expenses_Amount',
       'Salvage_or_Subrogation?', 'Coverage', 'Vehicle_identifier',
       'At_Fault?', 'Claim_Status_(close,_open,_reopen_etc)', 'Deductible',
       'Accident_area', 'number_of_claimants', 'Accident_Time'],
      dtype='object')


### training & testing

In [22]:
print(df_training.shape)
print('unique:',df_training['Policy_Number'].nunique()) #210763 unique Policy Number
df_training.head()

(210763, 2)
unique: 210763


,Policy_Number,Next_Premium
0,8e31b2f7864ddeac1be3a31766b0bf2c54908d37,4414
1,3f9f289dd1b20494a2fe4b1f0fe2d78333c97eda,8236
2,9864c955e499d1fa908f5dc7292054a232f01722,6153
3,92170afa649706baf10774dc0a46e67617c0d50d,12135
4,d16c11ef69f5f6bac02ed18fb1e372394563697f,4345


In [23]:
df_training[df_training['Next_Premium']==100]  # if Next_Premium = 100 -> testing_set

,Policy_Number,Next_Premium


In [24]:
print(df_testing.shape)
print('unique:',df_testing['Policy_Number'].nunique()) #140510 unique Policy Number
df_testing.head()

(140510, 2)
unique: 140510


,Policy_Number,Next_Premium
0,55789b8f86893761c9aa9e7bf17938e737decc68,100
1,b6df13a3384528ba6339c52b4fff7c149de68011,100
2,e112d926103147bcdcb6dab201b736185a3e2520,100
3,aa346fa4b1931d1c7a55f8e1bca40b0927dd65ac,100
4,39c4d5daaa791676ec5559c9066d7e8e8dfc51d7,100


In [25]:
a=df_training['Policy_Number'].nunique()/df_policy['Policy_Number'].nunique()*100
b=df_testing['Policy_Number'].nunique()/df_policy['Policy_Number'].nunique()*100
print('training set (pct)  %f '%a)
print('testing set (pct)  %f '%b)

training set (pct)  59.999772 
testing set (pct)  40.000228 


## 1. Combine policy & claim & training & testing

In [26]:
df_all= df_policy.merge(df_claim, on=df_policy.columns[0], how='left', suffixes=('','_Claim'))
print(df_all.columns)
print(df_all.shape)   # 60 items (overlap on Policy_Number only)

Index(['Policy_Number', 'Insured's_ID', 'Prior_Policy_Number', 'Cancellation',
       'Vehicle_identifier', 'Vehicle_Make_and_Model1',
       'Vehicle_Make_and_Model2', 'Manafactured_Year_and_Month',
       'Engine_Displacement_(Cubic_Centimeter)', 'Imported_or_Domestic_Car',
       'Coding_of_Vehicle_Branding_&_Type', 'qpt', 'fpt',
       'Main_Insurance_Coverage_Group', 'Insurance_Coverage',
       'Insured_Amount1', 'Insured_Amount2', 'Insured_Amount3',
       'Coverage_Deductible_if_applied', 'Premium',
       'Replacement_cost_of_insured_vehicle', 'Distribution_Channel',
       'Multiple_Products_with_TmNewa_(Yes_or_No?)', 'lia_class', 'plia_acc',
       'pdmg_acc', 'fassured', 'ibirth', 'fsex', 'fmarriage', 'aassured_zip',
       'iply_area', 'dbirth', 'fequipment1', 'fequipment2', 'fequipment3',
       'fequipment4', 'fequipment5', 'fequipment6', 'fequipment9',
       'nequipment9', 'Claim_Number', 'Nature_of_the_claim', 'Driver's_Gender',
       'Driver's_Relationship_with_Insu

In [27]:
df_all= df_all.merge(df_training, on=df_policy.columns[0], how='left')
df_all= df_all.merge(df_testing, on=df_policy.columns[0], how='left')
df_all['Next_Premium'] = df_all.apply(
    lambda row: row['Next_Premium_y'] if row['Next_Premium_y']==100 else row['Next_Premium_x'], axis = 1)
df_all=df_all.drop(['Next_Premium_x','Next_Premium_y'], axis =1)
print(df_all.shape) 
print(df_all.columns)

(1988927, 61)
Index(['Policy_Number', 'Insured's_ID', 'Prior_Policy_Number', 'Cancellation',
       'Vehicle_identifier', 'Vehicle_Make_and_Model1',
       'Vehicle_Make_and_Model2', 'Manafactured_Year_and_Month',
       'Engine_Displacement_(Cubic_Centimeter)', 'Imported_or_Domestic_Car',
       'Coding_of_Vehicle_Branding_&_Type', 'qpt', 'fpt',
       'Main_Insurance_Coverage_Group', 'Insurance_Coverage',
       'Insured_Amount1', 'Insured_Amount2', 'Insured_Amount3',
       'Coverage_Deductible_if_applied', 'Premium',
       'Replacement_cost_of_insured_vehicle', 'Distribution_Channel',
       'Multiple_Products_with_TmNewa_(Yes_or_No?)', 'lia_class', 'plia_acc',
       'pdmg_acc', 'fassured', 'ibirth', 'fsex', 'fmarriage', 'aassured_zip',
       'iply_area', 'dbirth', 'fequipment1', 'fequipment2', 'fequipment3',
       'fequipment4', 'fequipment5', 'fequipment6', 'fequipment9',
       'nequipment9', 'Claim_Number', 'Nature_of_the_claim', 'Driver's_Gender',
       'Driver's_Relation

In [28]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988927 entries, 0 to 1988926
Data columns (total 61 columns):
Policy_Number                                 object
Insured's_ID                                  object
Prior_Policy_Number                           object
Cancellation                                  object
Vehicle_identifier                            object
Vehicle_Make_and_Model1                       object
Vehicle_Make_and_Model2                       object
Manafactured_Year_and_Month                   int64
Engine_Displacement_(Cubic_Centimeter)        int64
Imported_or_Domestic_Car                      int64
Coding_of_Vehicle_Branding_&_Type             object
qpt                                           float64
fpt                                           object
Main_Insurance_Coverage_Group                 object
Insurance_Coverage                            object
Insured_Amount1                               int64
Insured_Amount2                           

### 1.1 check missing data

In [29]:
# % percentage
df_all.isnull().sum()/df_all.shape[0]*100   # about 22.7 declaire claim

Policy_Number                                  0.000000
Insured's_ID                                   0.000000
Prior_Policy_Number                           19.264709
Cancellation                                   0.000000
Vehicle_identifier                             0.025793
Vehicle_Make_and_Model1                        0.000000
Vehicle_Make_and_Model2                        0.000000
Manafactured_Year_and_Month                    0.000000
Engine_Displacement_(Cubic_Centimeter)         0.000000
Imported_or_Domestic_Car                       0.000000
Coding_of_Vehicle_Branding_&_Type              0.000000
qpt                                            0.000000
fpt                                            0.000000
Main_Insurance_Coverage_Group                  0.000000
Insurance_Coverage                             0.000000
Insured_Amount1                                0.000000
Insured_Amount2                                0.000000
Insured_Amount3                                0

In [30]:
### df_all.nunique()

# '''
# Policy_Number                                 351273  Encode
# Insured's_ID                                  299999  Encode
# Prior_Policy_Number                           281678  Encode
# Cancellation                                       2  Dummy
# Vehicle_identifier                            347882  Encode
# Vehicle_Make_and_Model1                          136  Encode (too many...)
# Vehicle_Make_and_Model2                         8112  Encode
# Manafactured_Year_and_Month                       49  Datetime
# Engine_Displacement_(Cubic_Centimeter)           768  no Change
# Imported_or_Domestic_Car                          10  Dummy
# Coding_of_Vehicle_Branding_&_Type               7405  Encode
# qpt                                               19  Dummy
# fpt                                                1  Dummy
# Main_Insurance_Coverage_Group                      3  Dummy
# Insurance_Coverage                                60  Dummy
# Insured_Amount1                                  111  no Change
# Insured_Amount2                                   89  no Change
# Insured_Amount3                                 3071  no Change
# Coverage_Deductible_if_applied                    53  no Change
# Premium                                        18238  no Change
# Replacement_cost_of_insured_vehicle             2947  no Change
# Distribution_Channel                             875  Encode (too many...)
# Multiple_Products_with_TmNewa_(Yes_or_No?)       179  no Change (dummy??)
# lia_class                                         21  no Change (dummy??)
# plia_acc                                          21  no Change (dummy??)
# pdmg_acc                                          13  no Change (dummy??)
# fassured                                           4  Dummy
# ibirth                                           918  Datetime
# fsex                                               4  Dummy
# fmarriage                                          4  Dummy 
# aassured_zip                                    1722  Encode (too many...)                                                                       ...  
# iply_area                                         22  Dummy
# dbirth                                           945  Datetime
# fequipment1                                        2  Dummy
# fequipment2                                        2  Dummy
# fequipment3                                        2  Dummy
# fequipment4                                        1  Dummy
# fequipment5                                        2  Dummy
# fequipment6                                        1  Dummy
# fequipment9                                        2  Dummy 
# nequipment9                                        6  Dummy
# ============ Claim  ========
# Claim_Number                                   41137  Encode
# Nature_of_the_claim                                2  Dummy
# Driver's_Gender                                    2  Dummy
# Driver's_Relationship_with_Insured                 7  Dummy
# DOB_of_Driver                                    799  Datetime
# Marital_Status_of_Driver                           2  Dummy
# Accident_Date                                     25  Datetime
# Cause_of_Loss                                     17  Dummy
# Paid_Loss_Amount                               17229  no Change 
# paid_Expenses_Amount                             799  no Change
# Salvage_or_Subrogation?                         2056  no Change
# Coverage                                          48  Encode (same as Insurance_Coverage!)
# Vehicle_identifier_Claim                       35714  Encode (same as Vehicle_identifier if applicable!)
# At_Fault?                                         23  no Change (dummy??)
# Claim_Status_(close,_open,_reopen_etc)             2  Dummy
# Deductible                                        67  no Change 
# Accident_area                                     22  Dummy  (same as aassured_zip?)
# number_of_claimants                               14  no Change (dummy??)
# Accident_Time                                     39  DateTime
# Next_Premium                                   22817  no Change
# '''

In [31]:
## fill na as 0 (Check explaination document, those 0 doesn't mean anything)

df= df_all
cols = ['Prior_Policy_Number','Vehicle_identifier','fsex','fmarriage','Claim_Number','Nature_of_the_claim','Policy_Number',
        "Driver's_Gender","Driver's_Relationship_with_Insured",'Marital_Status_of_Driver',
        'Cause_of_Loss','number_of_claimants']
for col in cols:
    df[col]=df[col].fillna(0)
    print(col, df[col].isnull().sum())


Prior_Policy_Number 0
Vehicle_identifier 0
fsex 0
fmarriage 0
Claim_Number 0
Nature_of_the_claim 0
Policy_Number 0
Driver's_Gender 0
Driver's_Relationship_with_Insured 0
Marital_Status_of_Driver 0
Cause_of_Loss 0
number_of_claimants 0


In [32]:
df_all = df

In [33]:
## fill na as -1 ( becasue 0 means something else)

df= df_all
cols = ['Paid_Loss_Amount','paid_Expenses_Amount','Salvage_or_Subrogation?', 'At_Fault?', 
        'Claim_Status_(close,_open,_reopen_etc)', 'Deductible']
for col in cols:
    df[col]=df[col].fillna(-1)
    print(col, df[col].isnull().sum())



Paid_Loss_Amount 0
paid_Expenses_Amount 0
Salvage_or_Subrogation? 0
At_Fault? 0
Claim_Status_(close,_open,_reopen_etc) 0
Deductible 0


In [34]:
df_all = df

In [35]:
##### Verify and drop 
df = df_all
### verify if 'Vehicle_identifier' = 'Vehicle_identifier_Claim'  (YES)
df_temp=df[['Policy_Number','Vehicle_identifier','Vehicle_identifier_Claim']]
df_temp2 = df_temp[df_temp['Vehicle_identifier_Claim'].notnull()] 
print(df_temp2.shape)
df_temp2[df_temp2['Vehicle_identifier']==df_temp2['Vehicle_identifier_Claim']]
print(df_temp2.shape)     

### verify if 'Insurance_Coverage'=='Coverage'    (YES)
df_temp=df[['Policy_Number','Insurance_Coverage','Coverage']]
df_temp2 = df_temp[df_temp['Coverage'].notnull()] 
print(df_temp2.shape)
df_temp2.head()
df_temp2[df_temp2['Insurance_Coverage']==df_temp2['Coverage']]
print(df_temp2.shape)  # 'Insurance_Coverage'=='Coverage' if there is value

### verify if 'Accident_area'=='iply_area'      (YES)
df_temp=df[['Policy_Number','Accident_area','iply_area']]
df_temp2 = df_temp[df_temp['Accident_area'].notnull()] 
print(df_temp2.shape)
df_temp2.head()
df_temp2[df_temp2['iply_area']==df_temp2['Accident_area']]
print(df_temp2.shape)  # 'Accident_area'=='iply_area'' if there is value

(449885, 3)
(449885, 3)
(452097, 3)
(452097, 3)
(452097, 3)
(452097, 3)


In [36]:
## drop 'Coverage' & 'Vehicle_identifier_Claim' & 'Accident_area
df= df_all
df = df.drop(['Coverage','Vehicle_identifier_Claim','Accident_area'], axis=1)
df.shape

(1988927, 58)

In [37]:
df_all = df

In [38]:
### fill NaN as (today)

# df= df_all
# cols = ['ibirth','dbirth','DOB_of_Driver','Accident_Date','Accident_Time']
# for col in cols:
#     df[col]=df[col].fillna(0)
#     print(col, df[col].isnull().sum())

In [39]:
df_all = df

In [40]:
# df_all.isnull().sum()   

In [41]:
df = df_all
df[df.columns[41:]].nunique()

Claim_Number                              41138
Nature_of_the_claim                           3
Driver's_Gender                               3
Driver's_Relationship_with_Insured            8
DOB_of_Driver                               798
Marital_Status_of_Driver                      3
Accident_Date                                24
Cause_of_Loss                                18
Paid_Loss_Amount                          17230
paid_Expenses_Amount                        800
Salvage_or_Subrogation?                    2057
At_Fault?                                    24
Claim_Status_(close,_open,_reopen_etc)        3
Deductible                                   68
number_of_claimants                          15
Accident_Time                                39
Next_Premium                              22817
dtype: int64

### 1.2 Label Encoder: convert anon id to No.

In [42]:
pd.set_option('display.max_columns', None)  ## show all columns
print(df_all.shape)
df_all.head()

(1988927, 58)


,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9,Claim_Number,Nature_of_the_claim,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,number_of_claimants,Accident_Time,Next_Premium
0,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16G,2000000,0,10000000,0,1551,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,2412.0
1,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,bb62ee2823e952188fbec8dbb63bb89ca14069e1,5.0,P,車責,16P,0,0,300000,0,1163,45.4,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,3,-0.1,0.0,1,08/1984,2,1,9a70776c743352cfcf688e52512673332e5e4007,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,2412.0
2,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16G,5000000,0,60000000,0,2031,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,3129.0
3,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,4d456fbc03a790f4912cc1ae8aea653e2614c194,5.0,P,車責,16P,0,0,500000,0,1098,59.9,5037717f120cf25bbeea6a42e62676bc5c911b12,0.0,1,-0.3,0.0,1,10/1975,2,1,e26973e6ee8ab9cd8cb3f207d1b90f00d2669eff,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,3129.0
4,1a1f5f582bf1ff23467352e8f2fd31acefddc654,Fcd3d3578e9ae55aa6140da432f69db2bd0c50fad,bd864927fc796b75c917ade554c0657d22b860bd,,3828f4a4c40ac91647919306d0de0d11043d89fe,de36049ef6a91e6dd1a031ab5501beca28cc3610,cca82bf061cfd670f460f9e4a1b9ca853378382b,2006,1341,10,f4c3b98bf06237d20aa5f6c73b0c5359019e1cf8,5.0,P,竊盜,05N,0,0,65000,10,149,39.9,86418116e42c87c4d555d363b14b11827f39b8c4,0.0,6,0.3,0.0,1,12/1989,2,2,9ffd1ae121c4f26fe7f0c45ecdc85fa6ac245bf0,c1dfd96eea8cc2b62785275bca38ac261256e278,12/1989,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,0.0


In [43]:
'''
encoded columns:

Policy_Number	保單號碼
Insured's_ID	被保險人代號(替代值)
Prior_Policy_Number	前保單號
Vehicle_identifier	車牌號碼                               ## same as vehicle_identifier
Vehicle_Make_and_Model1	廠牌名稱一                   
Vehicle_Make_and_Model2	廠牌名稱二
Coding_of_Vehicle_Branding_&_Type	廠牌車型代號
Distribution_Channel	主通路代號
aassured_zip	郵遞區號                            
Claim_Number	賠案號碼
Vehicle_identifier_Claim	車牌                           ## same as vehicle_identifier (already drop)

'''

"\nencoded columns:\n\nPolicy_Number\t保單號碼\nInsured's_ID\t被保險人代號(替代值)\nPrior_Policy_Number\t前保單號\nVehicle_identifier\t車牌號碼                               ## same as vehicle_identifier\nVehicle_Make_and_Model1\t廠牌名稱一                   \nVehicle_Make_and_Model2\t廠牌名稱二\nCoding_of_Vehicle_Branding_&_Type\t廠牌車型代號\nDistribution_Channel\t主通路代號\naassured_zip\t郵遞區號                            \nClaim_Number\t賠案號碼\nVehicle_identifier_Claim\t車牌                           ## same as vehicle_identifier (already drop)\n\n"

In [44]:
df = df_all

## do not include:  Vehicle_identifier_Claim	車牌  (map to Vehicle_identifier)
cols = ('Policy_Number',"Insured's_ID",'Prior_Policy_Number','Vehicle_identifier', 'Vehicle_Make_and_Model1',
       'Vehicle_Make_and_Model2','Coding_of_Vehicle_Branding_&_Type','Distribution_Channel','aassured_zip',
        'Distribution_Channel','aassured_zip','Claim_Number')


# process columns, apply LabelEncoder to categorical features
from sklearn.preprocessing import LabelEncoder
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(df[c].values)) 
    df[c] = lbl.transform(list(df[c].values))

# shape        
print('Shape of data: {}'.format(df.shape))
df.head()

Shape of data: (1988927, 58)


,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9,Claim_Number,Nature_of_the_claim,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,number_of_claimants,Accident_Time,Next_Premium
0,166470,48212,248050,,218187,59,2764,2010,1342,10,5426,5.0,P,車責,16G,2000000,0,10000000,0,1551,45.4,262,0.0,3,-0.1,0.0,1,08/1984,2,1,1043,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,2412.0
1,166470,48212,248050,,218187,59,2764,2010,1342,10,5426,5.0,P,車責,16P,0,0,300000,0,1163,45.4,262,0.0,3,-0.1,0.0,1,08/1984,2,1,1043,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,2412.0
2,134122,146227,179355,,148452,77,7004,2006,1794,10,2240,5.0,P,車責,16G,5000000,0,60000000,0,2031,59.9,262,0.0,1,-0.3,0.0,1,10/1975,2,1,1535,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,3129.0
3,134122,146227,179355,,148452,77,7004,2006,1794,10,2240,5.0,P,車責,16P,0,0,500000,0,1098,59.9,262,0.0,1,-0.3,0.0,1,10/1975,2,1,1535,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,3129.0
4,36048,110535,208666,,76333,114,6468,2006,1341,10,7059,5.0,P,竊盜,05N,0,0,65000,10,149,39.9,461,0.0,6,0.3,0.0,1,12/1989,2,2,1096,c1dfd96eea8cc2b62785275bca38ac261256e278,12/1989,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,0.0


In [45]:
### Already dropped
### encode 'Vehicle_identifier_Claim' (NaN: NaN, not NaN: 'Vehicle_identifier' )
# df = df_all
# condition1 = df['Vehicle_identifier_Claim'].isnull()
# condition2 = df['Vehicle_identifier_Claim'].notnull()

# df['Vehicle_identifier_Claim'][condition1]=df['Vehicle_identifier_Claim']
# df['Vehicle_identifier_Claim'][condition2]=df['Vehicle_identifier']

#df[['Policy_Number','Vehicle_identifier','Vehicle_identifier_Claim']].sample(5)

### 1.3 Get dummy

In [46]:

'''
dummy columns:

Cancellation                   非繼承自前保單	類別	Y: 是 、空白: 否(N)
Imported_or_Domestic_Car       車系代號	類別	參照：車系代號Sheet
qpt                            乘載數量	數值
fpt                            乘載單位	類別	P:載客 T:噸
Main_Insurance_Coverage_Group  險種分類	類別	參照:險種分類及自負額說明Sheet
Insurance_Coverage    ####     險種代碼	類別	參照:險種分類及自負額說明Sheet
fassured                       被保險人性質	類別	"1:國內自然人 2:國內法人 3:國外自然人 4:國外法人 5:自然人ID有誤 6:法人ID有誤"
fsex                           性別	類別	1:男 2:女 空白:法人(0)
fmarriage                      婚姻狀況	類別	1:已婚 2:未婚
iply_area             ####     承保地區代號(分公司)	類別
fequipment1                    配備-音響註記	類別	1:是 0:否
fequipment2                    配備-車箱註記	類別	1:是 0:否
fequipment3                    配備-冷凍機組註記	類別	1:是 0:否
fequipment4                    配備-車框、車斗註記	類別	1:是 0:否
fequipment5                    配備-昇降系統註記	類別	1:是 0:否
fequipment6                    配備-電動車電池註記	類別	1:是 0:否
fequipment9                    配備-其他	類別	1:是 0:否
nequipment9                    配備-其他說明	類別	1:是 0:否
Nature_of_the_claim            賠案性質	類別	1.賠付 2.追償
Driver's_Gender                肇事駕駛性別	類別	1:男 2:女 空白:法人
Driver's_Relationship_with_Insured 與被保險人關係	類別	"1:被保險人本人 2:被保險人親友 3:被保險人員工 4:租用被保險車輛 5:其他, 6:配偶, 7:子女"
Marital_Status_of_Driver       肇事駕駛婚姻	類別	1:已婚 2:未婚
Cause_of_Loss                  出險原因	類別
Coverage             ###       險種代號	類別  same as insurance_coverage (Already Dropped)
Accident_area        ####      出險地區	類別  same as iply (AlreadyDrop)
'''

'\ndummy columns:\n\nCancellation                   非繼承自前保單\t類別\tY: 是 、空白: 否(N)\nImported_or_Domestic_Car       車系代號\t類別\t參照：車系代號Sheet\nqpt                            乘載數量\t數值\nfpt                            乘載單位\t類別\tP:載客 T:噸\nMain_Insurance_Coverage_Group  險種分類\t類別\t參照:險種分類及自負額說明Sheet\nInsurance_Coverage    ####     險種代碼\t類別\t參照:險種分類及自負額說明Sheet\nfassured                       被保險人性質\t類別\t"1:國內自然人 2:國內法人 3:國外自然人 4:國外法人 5:自然人ID有誤 6:法人ID有誤"\nfsex                           性別\t類別\t1:男 2:女 空白:法人(0)\nfmarriage                      婚姻狀況\t類別\t1:已婚 2:未婚\niply_area             ####     承保地區代號(分公司)\t類別\nfequipment1                    配備-音響註記\t類別\t1:是 0:否\nfequipment2                    配備-車箱註記\t類別\t1:是 0:否\nfequipment3                    配備-冷凍機組註記\t類別\t1:是 0:否\nfequipment4                    配備-車框、車斗註記\t類別\t1:是 0:否\nfequipment5                    配備-昇降系統註記\t類別\t1:是 0:否\nfequipment6                    配備-電動車電池註記\t類別\t1:是 0:否\nfequipment9                    配備-其他\t類別\t1:是 0:否\nnequipment9        

In [47]:
## Label cancellation from {'Y', ' '}-> {'Y', 'N'} get dummy later
df= df_all
print(set(df['Cancellation']))
input1 = ' '
replace1 = 'N'
df['Cancellation']=df['Cancellation'].replace(input1, replace1) 
df.head()

{' ', 'Y'}


,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9,Claim_Number,Nature_of_the_claim,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,number_of_claimants,Accident_Time,Next_Premium
0,166470,48212,248050,N,218187,59,2764,2010,1342,10,5426,5.0,P,車責,16G,2000000,0,10000000,0,1551,45.4,262,0.0,3,-0.1,0.0,1,08/1984,2,1,1043,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,2412.0
1,166470,48212,248050,N,218187,59,2764,2010,1342,10,5426,5.0,P,車責,16P,0,0,300000,0,1163,45.4,262,0.0,3,-0.1,0.0,1,08/1984,2,1,1043,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,2412.0
2,134122,146227,179355,N,148452,77,7004,2006,1794,10,2240,5.0,P,車責,16G,5000000,0,60000000,0,2031,59.9,262,0.0,1,-0.3,0.0,1,10/1975,2,1,1535,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,3129.0
3,134122,146227,179355,N,148452,77,7004,2006,1794,10,2240,5.0,P,車責,16P,0,0,500000,0,1098,59.9,262,0.0,1,-0.3,0.0,1,10/1975,2,1,1535,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,3129.0
4,36048,110535,208666,N,76333,114,6468,2006,1341,10,7059,5.0,P,竊盜,05N,0,0,65000,10,149,39.9,461,0.0,6,0.3,0.0,1,12/1989,2,2,1096,c1dfd96eea8cc2b62785275bca38ac261256e278,12/1989,0,0,0,0,0,0,0,,0,0.0,0.0,0.0,NaN,0.0,NaN,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,NaN,0.0


In [48]:
df_all = df

In [49]:
### Getting dummy categorical features:

In [50]:
df = df_all
segment=['Cancellation','Imported_or_Domestic_Car','qpt','fpt','Main_Insurance_Coverage_Group','Insurance_Coverage',
         'fassured', 'fsex','fmarriage','iply_area', 'fequipment1','fequipment2','fequipment3','fequipment4',
         'fequipment5','fequipment6','fequipment9','nequipment9','Nature_of_the_claim',"Driver's_Gender",
         "Driver's_Relationship_with_Insured", 'Marital_Status_of_Driver', 'Cause_of_Loss']
df_dummy = pd.get_dummies(df, columns=segment)
print(df_dummy.shape)
df_dummy.head()

(1988927, 217)


,Policy_Number,Insured's_ID,Prior_Policy_Number,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Coding_of_Vehicle_Branding_&_Type,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,ibirth,aassured_zip,dbirth,Claim_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,number_of_claimants,Accident_Time,Next_Premium,Cancellation_N,Cancellation_Y,Imported_or_Domestic_Car_10,Imported_or_Domestic_Car_20,Imported_or_Domestic_Car_21,Imported_or_Domestic_Car_22,Imported_or_Domestic_Car_23,Imported_or_Domestic_Car_24,Imported_or_Domestic_Car_30,Imported_or_Domestic_Car_40,Imported_or_Domestic_Car_50,Imported_or_Domestic_Car_90,qpt_2.0,qpt_3.0,qpt_4.0,qpt_5.0,qpt_6.0,qpt_7.0,qpt_8.0,qpt_9.0,qpt_11.0,qpt_12.0,qpt_13.0,qpt_17.0,qpt_18.0,qpt_19.0,qpt_20.0,qpt_21.0,qpt_22.0,qpt_23.0,qpt_24.0,fpt_P,Main_Insurance_Coverage_Group_竊盜,Main_Insurance_Coverage_Group_車損,Main_Insurance_Coverage_Group_車責,Insurance_Coverage_00I,Insurance_Coverage_01A,Insurance_Coverage_01J,Insurance_Coverage_02K,Insurance_Coverage_03L,Insurance_Coverage_04M,Insurance_Coverage_05E,Insurance_Coverage_05N,Insurance_Coverage_06F,Insurance_Coverage_07P,Insurance_Coverage_08H,Insurance_Coverage_09@,Insurance_Coverage_09I,Insurance_Coverage_10A,Insurance_Coverage_12L,Insurance_Coverage_14E,Insurance_Coverage_14N,Insurance_Coverage_15F,Insurance_Coverage_15O,Insurance_Coverage_16G,Insurance_Coverage_16P,Insurance_Coverage_18@,Insurance_Coverage_18I,Insurance_Coverage_20B,Insurance_Coverage_20K,Insurance_Coverage_25G,Insurance_Coverage_26H,Insurance_Coverage_27I,Insurance_Coverage_29B,Insurance_Coverage_29K,Insurance_Coverage_32N,Insurance_Coverage_33F,Insurance_Coverage_33O,Insurance_Coverage_34P,Insurance_Coverage_35H,Insurance_Coverage_36I,Insurance_Coverage_37J,Insurance_Coverage_40M,Insurance_Coverage_41E,Insurance_Coverage_41N,Insurance_Coverage_42F,Insurance_Coverage_45@,Insurance_Coverage_46A,Insurance_Coverage_47B,Insurance_Coverage_51O,Insurance_Coverage_55J,Insurance_Coverage_56B,Insurance_Coverage_56K,Insurance_Coverage_57C,Insurance_Coverage_57L,Insurance_Coverage_65K,Insurance_Coverage_66C,Insurance_Coverage_66L,Insurance_Coverage_67D,Insurance_Coverage_68E,Insurance_Coverage_68N,Insurance_Coverage_70G,Insurance_Coverage_70P,Insurance_Coverage_71H,Insurance_Coverage_72@,fassured_1,fassured_2,fassured_3,fassured_6,fsex_0,fsex_,fsex_1,fsex_2,fmarriage_0,fmarriage_,fmarriage_1,fmarriage_2,iply_area_0286dd552c9bea9a69ecb3759e7b94777635514b,iply_area_17ba0791499db908433b80f37c5fbc89b870084b,iply_area_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,iply_area_2d0c8af807ef45ac17cafb2973d866ba8f38caa9,iply_area_356a192b7913b04c54574d18c28d46e6395428ab,iply_area_4cd66dfabbd964f8c6c4414b07cdb45dae692e19,iply_area_4d89d294cd4ca9f2ca57dc24a53ffb3ef5303122,iply_area_511a418e72591eb7e33f703f04c3fa16df6c90bd,iply_area_59129aacfb6cebbe2c52f30ef3424209f7252e82,iply_area_6c1e671f9af5b46d9c1a52067bdf0e53685674f7,iply_area_761f22b2c1593d0bb87e0b606f990ba4974706de,iply_area_76546f9a641ede2beab506b96df1688d889e629a,iply_area_7b52009b64fd0a2a49e6d8a939753077792b0554,iply_area_92cfceb39d57d914ed8b14d0e37643de0797ae56,iply_area_98fbc42faedc02492397cb5962ea3a3ffc0a9243,iply_area_a17554a0d2b15a664c0e73900184544f19e70227,iply_area_bd307a3ec329e10a2cff8fb87480823da114f8f4,iply_area_c1dfd96eea8cc2b62785275bca38ac261256e278,iply_area_c66c65175fecc3103b3b587be9b5b230889c8628,iply_area_f1abd670358e036c31296e66b3b66c382ac00812,iply_area_fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,iply_area_fb644351560d8296fe6da332236b1f8d61b2828a,fequipment1_0,fequipment1_1,fequipment2_0,fequipment2_1,fequipment3_0,fequipment3_1,fequipment4_0,fequipment5_0,fequipment5_1,fequip

### 1.4 convert object (date) to datetime format

In [51]:
## check date format
date_cols=['ibirth','dbirth','DOB_of_Driver','Accident_Date','Accident_Time']


In [52]:
df_dummy[date_cols].info()
"""
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988927 entries, 0 to 1988926
Data columns (total 5 columns):
ibirth           object
dbirth           object               # object... (selected index repalce with ibirth)
DOB_of_Driver    object
Accident_Date    object
Accident_Time    object
dtypes: datetime64[ns](4), object(1)
memory usage: 91.0+ MB
"""

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988927 entries, 0 to 1988926
Data columns (total 5 columns):
ibirth           object
dbirth           object
DOB_of_Driver    object
Accident_Date    object
Accident_Time    object
dtypes: object(5)
memory usage: 91.0+ MB


"\n<class 'pandas.core.frame.DataFrame'>\nInt64Index: 1988927 entries, 0 to 1988926\nData columns (total 5 columns):\nibirth           object\ndbirth           object               # object... (selected index repalce with ibirth)\nDOB_of_Driver    object\nAccident_Date    object\nAccident_Time    object\ndtypes: datetime64[ns](4), object(1)\nmemory usage: 91.0+ MB\n"

In [53]:
df_dummy[date_cols].sample(10)  ## 77% missing data

,ibirth,dbirth,DOB_of_Driver,Accident_Date,Accident_Time
184577,NaN,NaN,NaN,NaN,NaN
1023565,11/1978,11/1978,NaN,NaN,NaN
1153165,11/1979,11/1979,11/1979,2015/07,07:00
1429038,07/1981,07/1981,NaN,NaN,NaN
604308,02/1957,02/1957,NaN,NaN,NaN
196833,04/1984,04/1984,04/1984,2015/06,14:00
1238821,10/1975,10/1975,NaN,NaN,NaN
499495,10/1986,10/1986,05/1984,2016/02,13:00
1753637,04/1962,04/1962,NaN,NaN,NaN
602616,NaN,NaN,12/1983,2015/03,10:00


In [54]:
type(df_dummy['Accident_Time'][1000000])

str

In [55]:
### fill NaN as (2020)
df= df_dummy

cols = ['Accident_Time']
for col in cols:
    print(df[col].isnull().sum())
    df[col]=df[col].fillna(str('00:00'))
    print(col, df[col].isnull().sum())

1536830
Accident_Time 0


In [56]:
type(df['Accident_Time'][1])
df['Accident_Time'].sample(10)

18032      00:00
62859      13:00
1269126    00:00
265704     00:00
107911     00:00
1667599    00:00
1742858    00:00
574959     00:00
164938     00:00
1374049    00:00
Name: Accident_Time, dtype: object

In [57]:
set(df['Accident_Time'])

{'00:00',
 '01:00',
 '02:00',
 '03:00',
 '04:00',
 '05:00',
 '06:00',
 '07:00',
 '08:00',
 '09:00',
 '09:22',
 '09:52',
 '10:00',
 '11:00',
 '11:53',
 '12:00',
 '12:02',
 '12:50',
 '13:00',
 '13:12',
 '14:00',
 '14:06',
 '15:00',
 '15:14',
 '15:15',
 '16:00',
 '16:48',
 '17:00',
 '17:30',
 '17:38',
 '17:43',
 '18:00',
 '18:01',
 '18:32',
 '19:00',
 '20:00',
 '21:00',
 '22:00',
 '23:00'}

In [58]:
### fill NaN as (2020)

cols = ['ibirth','dbirth','DOB_of_Driver']

for col in cols:
    print(df[col].isnull().sum())
    df[col]=df[col].fillna('09/2018')
    print(col, df[col].isnull().sum())

cols = ['Accident_Date']
for col in cols:
    print(df[col].isnull().sum())
    df[col]=df[col].fillna('2018/09')
    print(col, df[col].isnull().sum())

df.isnull().sum().sum()    
df_dummy= df

374483
ibirth 0
306676
dbirth 0
1536830
DOB_of_Driver 0
1536830
Accident_Date 0


In [59]:
df_dummy[date_cols].sample(10)

,ibirth,dbirth,DOB_of_Driver,Accident_Date,Accident_Time
514838,03/1981,03/1981,09/2018,2018/09,00:00
72004,11/1981,11/1981,09/2018,2018/09,00:00
1467069,08/1976,08/1976,09/2018,2018/09,00:00
1560801,04/1976,04/1976,09/2018,2018/09,00:00
42930,10/1957,10/1957,10/1957,2016/01,18:00
1361738,07/1975,07/1975,09/2018,2018/09,00:00
1935895,08/1977,08/1977,09/2018,2018/09,00:00
76332,07/1967,07/1967,07/1967,2015/07,17:00
1254037,07/1975,07/1975,07/1975,2015/10,19:00
1266277,01/1965,01/1965,09/2018,2018/09,00:00


In [60]:
df= df_dummy

date_col={'ibirth','dbirth','DOB_of_Driver'}
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%m/%Y', errors='ignore')  #'%Y-%m-%d %H:%M:%S'   #errors='coerce'

date_col={'Accident_Date'}
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%Y/%m',errors='ignore')  #'%Y-%m-%d %H:%M:%S'

date_col={'Accident_Time'}
for col in date_col:
    df[col] =  pd.to_datetime(df[col], format='%H:%M',errors='ignore') #'%Y-%m-%d %H:%M:%S'

df_dummy_date = df

In [61]:
df_dummy_date[['ibirth', 'dbirth' ,'DOB_of_Driver', 'Accident_Date', 'Accident_Time']].info(verbose= True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988927 entries, 0 to 1988926
Data columns (total 5 columns):
ibirth           datetime64[ns]
dbirth           datetime64[ns]
DOB_of_Driver    datetime64[ns]
Accident_Date    datetime64[ns]
Accident_Time    datetime64[ns]
dtypes: datetime64[ns](5)
memory usage: 91.0 MB


### 1.4.1 Extract Features from datetime data then drop datetime64 columns

In [62]:
df_dummy_date[['ibirth', 'dbirth' ,'DOB_of_Driver', 'Accident_Date', 'Accident_Time']].loc[932405]

ibirth          1974-07-01 00:00:00
dbirth          1974-07-01 00:00:00
DOB_of_Driver   1974-07-01 00:00:00
Accident_Date   2015-10-01 00:00:00
Accident_Time   1900-01-01 14:00:00
Name: 932405, dtype: datetime64[ns]

In [63]:
df_dummy_date[['ibirth', 'dbirth' ,'DOB_of_Driver', 'Accident_Date', 'Accident_Time']].sample(10)

,ibirth,dbirth,DOB_of_Driver,Accident_Date,Accident_Time
427619,1976-09-01,1976-09-01,2018-09-01,2018-09-01,1900-01-01 00:00:00
1398238,1977-03-01,1977-03-01,1977-03-01,2016-01-01,1900-01-01 17:00:00
616727,1985-01-01,1985-01-01,1985-01-01,2015-05-01,1900-01-01 07:00:00
1668761,1971-12-01,1971-12-01,2018-09-01,2018-09-01,1900-01-01 00:00:00
1062136,2018-09-01,2018-09-01,2018-09-01,2018-09-01,1900-01-01 00:00:00
902402,1963-04-01,1963-04-01,2018-09-01,2018-09-01,1900-01-01 00:00:00
1957022,1956-06-01,1956-06-01,2018-09-01,2018-09-01,1900-01-01 00:00:00
520628,1952-11-01,1952-11-01,2018-09-01,2018-09-01,1900-01-01 00:00:00
1246941,1960-06-01,1960-06-01,2018-09-01,2018-09-01,1900-01-01 00:00:00
342691,2018-09-01,2018-09-01,1952-10-01,2016-07-01,1900-01-01 16:00:00


In [64]:
## Today
today = datetime.date.today()
daysdiff = today-df_dummy_date['ibirth']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
components.head()

,days,hours,minutes,seconds,milliseconds,microseconds,nanoseconds
0,12413,0,0,0,0,0,0
1,12413,0,0,0,0,0,0
2,15640,0,0,0,0,0,0
3,15640,0,0,0,0,0,0
4,10465,0,0,0,0,0,0


In [65]:
years = components.days/365
df_dummy_date['iYearsOld'] = years
df_dummy_date.head()

,Policy_Number,Insured's_ID,Prior_Policy_Number,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Coding_of_Vehicle_Branding_&_Type,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,ibirth,aassured_zip,dbirth,Claim_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,number_of_claimants,Accident_Time,Next_Premium,Cancellation_N,Cancellation_Y,Imported_or_Domestic_Car_10,Imported_or_Domestic_Car_20,Imported_or_Domestic_Car_21,Imported_or_Domestic_Car_22,Imported_or_Domestic_Car_23,Imported_or_Domestic_Car_24,Imported_or_Domestic_Car_30,Imported_or_Domestic_Car_40,Imported_or_Domestic_Car_50,Imported_or_Domestic_Car_90,qpt_2.0,qpt_3.0,qpt_4.0,qpt_5.0,qpt_6.0,qpt_7.0,qpt_8.0,qpt_9.0,qpt_11.0,qpt_12.0,qpt_13.0,qpt_17.0,qpt_18.0,qpt_19.0,qpt_20.0,qpt_21.0,qpt_22.0,qpt_23.0,qpt_24.0,fpt_P,Main_Insurance_Coverage_Group_竊盜,Main_Insurance_Coverage_Group_車損,Main_Insurance_Coverage_Group_車責,Insurance_Coverage_00I,Insurance_Coverage_01A,Insurance_Coverage_01J,Insurance_Coverage_02K,Insurance_Coverage_03L,Insurance_Coverage_04M,Insurance_Coverage_05E,Insurance_Coverage_05N,Insurance_Coverage_06F,Insurance_Coverage_07P,Insurance_Coverage_08H,Insurance_Coverage_09@,Insurance_Coverage_09I,Insurance_Coverage_10A,Insurance_Coverage_12L,Insurance_Coverage_14E,Insurance_Coverage_14N,Insurance_Coverage_15F,Insurance_Coverage_15O,Insurance_Coverage_16G,Insurance_Coverage_16P,Insurance_Coverage_18@,Insurance_Coverage_18I,Insurance_Coverage_20B,Insurance_Coverage_20K,Insurance_Coverage_25G,Insurance_Coverage_26H,Insurance_Coverage_27I,Insurance_Coverage_29B,Insurance_Coverage_29K,Insurance_Coverage_32N,Insurance_Coverage_33F,Insurance_Coverage_33O,Insurance_Coverage_34P,Insurance_Coverage_35H,Insurance_Coverage_36I,Insurance_Coverage_37J,Insurance_Coverage_40M,Insurance_Coverage_41E,Insurance_Coverage_41N,Insurance_Coverage_42F,Insurance_Coverage_45@,Insurance_Coverage_46A,Insurance_Coverage_47B,Insurance_Coverage_51O,Insurance_Coverage_55J,Insurance_Coverage_56B,Insurance_Coverage_56K,Insurance_Coverage_57C,Insurance_Coverage_57L,Insurance_Coverage_65K,Insurance_Coverage_66C,Insurance_Coverage_66L,Insurance_Coverage_67D,Insurance_Coverage_68E,Insurance_Coverage_68N,Insurance_Coverage_70G,Insurance_Coverage_70P,Insurance_Coverage_71H,Insurance_Coverage_72@,fassured_1,fassured_2,fassured_3,fassured_6,fsex_0,fsex_,fsex_1,fsex_2,fmarriage_0,fmarriage_,fmarriage_1,fmarriage_2,iply_area_0286dd552c9bea9a69ecb3759e7b94777635514b,iply_area_17ba0791499db908433b80f37c5fbc89b870084b,iply_area_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,iply_area_2d0c8af807ef45ac17cafb2973d866ba8f38caa9,iply_area_356a192b7913b04c54574d18c28d46e6395428ab,iply_area_4cd66dfabbd964f8c6c4414b07cdb45dae692e19,iply_area_4d89d294cd4ca9f2ca57dc24a53ffb3ef5303122,iply_area_511a418e72591eb7e33f703f04c3fa16df6c90bd,iply_area_59129aacfb6cebbe2c52f30ef3424209f7252e82,iply_area_6c1e671f9af5b46d9c1a52067bdf0e53685674f7,iply_area_761f22b2c1593d0bb87e0b606f990ba4974706de,iply_area_76546f9a641ede2beab506b96df1688d889e629a,iply_area_7b52009b64fd0a2a49e6d8a939753077792b0554,iply_area_92cfceb39d57d914ed8b14d0e37643de0797ae56,iply_area_98fbc42faedc02492397cb5962ea3a3ffc0a9243,iply_area_a17554a0d2b15a664c0e73900184544f19e70227,iply_area_bd307a3ec329e10a2cff8fb87480823da114f8f4,iply_area_c1dfd96eea8cc2b62785275bca38ac261256e278,iply_area_c66c65175fecc3103b3b587be9b5b230889c8628,iply_area_f1abd670358e036c31296e66b3b66c382ac00812,iply_area_fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,iply_area_fb644351560d8296fe6da332236b1f8d61b2828a,fequipment1_0,fequipment1_1,fequipment2_0,fequipment2_1,fequipment3_0,fequipment3_1,fequipment4_0,fequipment5_0,fequipment5_1,fequip

In [66]:
today = datetime.date.today()
daysdiff = today-df_dummy_date['dbirth']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
components.head()

,days,hours,minutes,seconds,milliseconds,microseconds,nanoseconds
0,12413,0,0,0,0,0,0
1,12413,0,0,0,0,0,0
2,15640,0,0,0,0,0,0
3,15640,0,0,0,0,0,0
4,10465,0,0,0,0,0,0


In [67]:
years = components.days/365
df_dummy_date['dYearsOld'] = years
df_dummy_date['dYearsOld'].head()

0    34.008219
1    34.008219
2    42.849315
3    42.849315
4    28.671233
Name: dYearsOld, dtype: float64

In [68]:
today = datetime.date.today()
daysdiff = today-df_dummy_date['DOB_of_Driver']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
components.head()
years = components.days/365
df_dummy_date['DriverYearsOld'] = years
df_dummy_date['DriverYearsOld'].head()

0   -0.09863
1   -0.09863
2   -0.09863
3   -0.09863
4   -0.09863
Name: DriverYearsOld, dtype: float64

In [69]:
set(df_claim['Accident_Date'])  # between 2015/01 -> 2016/12    24months

{'2015/01',
 '2015/02',
 '2015/03',
 '2015/04',
 '2015/05',
 '2015/06',
 '2015/07',
 '2015/08',
 '2015/09',
 '2015/10',
 '2015/11',
 '2015/12',
 '2016/01',
 '2016/02',
 '2016/03',
 '2016/04',
 '2016/05',
 '2016/06',
 '2016/07',
 '2016/08',
 '2016/09',
 '2016/10',
 '2016/11',
 '2016/12'}

In [70]:
daysdiff = today-df_dummy_date['Accident_Date']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
months = components.days/30
df_dummy_date['Accident_Month_ago'] = months
df_dummy_date['Accident_Month_ago'].sample(10)

32872      39.433333
1567978    -1.200000
575597     -1.200000
917336     36.366667
1799170    -1.200000
402771     -1.200000
1666114    -1.200000
80765      -1.200000
1404833    24.166667
1721404    -1.200000
Name: Accident_Month_ago, dtype: float64

In [71]:
from datetime import datetime, date
df_dummy_date['Acc_DayOfWeek'] = df_dummy_date.apply(lambda row: row['Accident_Date'].weekday(), axis =1)
df_dummy_date['Acc_DayOfWeek'][:5]  

0    5
1    5
2    5
3    5
4    5
Name: Acc_DayOfWeek, dtype: int64

In [72]:
daysdiff = today-df_dummy_date['Accident_Time']
components = daysdiff.dt.components #  https://stackoverflow.com/questions/18215317/extracting-days-from-a-numpy-timedelta64-value
hours = components.hours
df_dummy_date['Accident_Hour'] = hours
df_dummy_date['Accident_Hour'].sample(10)

1013497     0
1984050    17
609562      0
220055      7
1936146     0
131345      0
867634      0
388493     14
51488       0
1834834     0
Name: Accident_Hour, dtype: int64

In [73]:
df_dummy_date.head()

,Policy_Number,Insured's_ID,Prior_Policy_Number,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Coding_of_Vehicle_Branding_&_Type,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,ibirth,aassured_zip,dbirth,Claim_Number,DOB_of_Driver,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,number_of_claimants,Accident_Time,Next_Premium,Cancellation_N,Cancellation_Y,Imported_or_Domestic_Car_10,Imported_or_Domestic_Car_20,Imported_or_Domestic_Car_21,Imported_or_Domestic_Car_22,Imported_or_Domestic_Car_23,Imported_or_Domestic_Car_24,Imported_or_Domestic_Car_30,Imported_or_Domestic_Car_40,Imported_or_Domestic_Car_50,Imported_or_Domestic_Car_90,qpt_2.0,qpt_3.0,qpt_4.0,qpt_5.0,qpt_6.0,qpt_7.0,qpt_8.0,qpt_9.0,qpt_11.0,qpt_12.0,qpt_13.0,qpt_17.0,qpt_18.0,qpt_19.0,qpt_20.0,qpt_21.0,qpt_22.0,qpt_23.0,qpt_24.0,fpt_P,Main_Insurance_Coverage_Group_竊盜,Main_Insurance_Coverage_Group_車損,Main_Insurance_Coverage_Group_車責,Insurance_Coverage_00I,Insurance_Coverage_01A,Insurance_Coverage_01J,Insurance_Coverage_02K,Insurance_Coverage_03L,Insurance_Coverage_04M,Insurance_Coverage_05E,Insurance_Coverage_05N,Insurance_Coverage_06F,Insurance_Coverage_07P,Insurance_Coverage_08H,Insurance_Coverage_09@,Insurance_Coverage_09I,Insurance_Coverage_10A,Insurance_Coverage_12L,Insurance_Coverage_14E,Insurance_Coverage_14N,Insurance_Coverage_15F,Insurance_Coverage_15O,Insurance_Coverage_16G,Insurance_Coverage_16P,Insurance_Coverage_18@,Insurance_Coverage_18I,Insurance_Coverage_20B,Insurance_Coverage_20K,Insurance_Coverage_25G,Insurance_Coverage_26H,Insurance_Coverage_27I,Insurance_Coverage_29B,Insurance_Coverage_29K,Insurance_Coverage_32N,Insurance_Coverage_33F,Insurance_Coverage_33O,Insurance_Coverage_34P,Insurance_Coverage_35H,Insurance_Coverage_36I,Insurance_Coverage_37J,Insurance_Coverage_40M,Insurance_Coverage_41E,Insurance_Coverage_41N,Insurance_Coverage_42F,Insurance_Coverage_45@,Insurance_Coverage_46A,Insurance_Coverage_47B,Insurance_Coverage_51O,Insurance_Coverage_55J,Insurance_Coverage_56B,Insurance_Coverage_56K,Insurance_Coverage_57C,Insurance_Coverage_57L,Insurance_Coverage_65K,Insurance_Coverage_66C,Insurance_Coverage_66L,Insurance_Coverage_67D,Insurance_Coverage_68E,Insurance_Coverage_68N,Insurance_Coverage_70G,Insurance_Coverage_70P,Insurance_Coverage_71H,Insurance_Coverage_72@,fassured_1,fassured_2,fassured_3,fassured_6,fsex_0,fsex_,fsex_1,fsex_2,fmarriage_0,fmarriage_,fmarriage_1,fmarriage_2,iply_area_0286dd552c9bea9a69ecb3759e7b94777635514b,iply_area_17ba0791499db908433b80f37c5fbc89b870084b,iply_area_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,iply_area_2d0c8af807ef45ac17cafb2973d866ba8f38caa9,iply_area_356a192b7913b04c54574d18c28d46e6395428ab,iply_area_4cd66dfabbd964f8c6c4414b07cdb45dae692e19,iply_area_4d89d294cd4ca9f2ca57dc24a53ffb3ef5303122,iply_area_511a418e72591eb7e33f703f04c3fa16df6c90bd,iply_area_59129aacfb6cebbe2c52f30ef3424209f7252e82,iply_area_6c1e671f9af5b46d9c1a52067bdf0e53685674f7,iply_area_761f22b2c1593d0bb87e0b606f990ba4974706de,iply_area_76546f9a641ede2beab506b96df1688d889e629a,iply_area_7b52009b64fd0a2a49e6d8a939753077792b0554,iply_area_92cfceb39d57d914ed8b14d0e37643de0797ae56,iply_area_98fbc42faedc02492397cb5962ea3a3ffc0a9243,iply_area_a17554a0d2b15a664c0e73900184544f19e70227,iply_area_bd307a3ec329e10a2cff8fb87480823da114f8f4,iply_area_c1dfd96eea8cc2b62785275bca38ac261256e278,iply_area_c66c65175fecc3103b3b587be9b5b230889c8628,iply_area_f1abd670358e036c31296e66b3b66c382ac00812,iply_area_fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,iply_area_fb644351560d8296fe6da332236b1f8d61b2828a,fequipment1_0,fequipment1_1,fequipment2_0,fequipment2_1,fequipment3_0,fequipment3_1,fequipment4_0,fequipment5_0,fequipment5_1,fequip

In [74]:
df_dummy_date[date_cols].head()

,ibirth,dbirth,DOB_of_Driver,Accident_Date,Accident_Time
0,1984-08-01,1984-08-01,2018-09-01,2018-09-01,1900-01-01
1,1984-08-01,1984-08-01,2018-09-01,2018-09-01,1900-01-01
2,1975-10-01,1975-10-01,2018-09-01,2018-09-01,1900-01-01
3,1975-10-01,1975-10-01,2018-09-01,2018-09-01,1900-01-01
4,1989-12-01,1989-12-01,2018-09-01,2018-09-01,1900-01-01


In [75]:
drop_date_cols = ['ibirth', 'dbirth', 'DOB_of_Driver','Accident_Date', 'Accident_Time']
df_dummy_date = df_dummy_date.drop (drop_date_cols, axis = 1)

In [76]:
print(df_dummy_date.shape)
df_dummy_date.head()

(1988927, 218)


,Policy_Number,Insured's_ID,Prior_Policy_Number,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Coding_of_Vehicle_Branding_&_Type,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,aassured_zip,Claim_Number,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,number_of_claimants,Next_Premium,Cancellation_N,Cancellation_Y,Imported_or_Domestic_Car_10,Imported_or_Domestic_Car_20,Imported_or_Domestic_Car_21,Imported_or_Domestic_Car_22,Imported_or_Domestic_Car_23,Imported_or_Domestic_Car_24,Imported_or_Domestic_Car_30,Imported_or_Domestic_Car_40,Imported_or_Domestic_Car_50,Imported_or_Domestic_Car_90,qpt_2.0,qpt_3.0,qpt_4.0,qpt_5.0,qpt_6.0,qpt_7.0,qpt_8.0,qpt_9.0,qpt_11.0,qpt_12.0,qpt_13.0,qpt_17.0,qpt_18.0,qpt_19.0,qpt_20.0,qpt_21.0,qpt_22.0,qpt_23.0,qpt_24.0,fpt_P,Main_Insurance_Coverage_Group_竊盜,Main_Insurance_Coverage_Group_車損,Main_Insurance_Coverage_Group_車責,Insurance_Coverage_00I,Insurance_Coverage_01A,Insurance_Coverage_01J,Insurance_Coverage_02K,Insurance_Coverage_03L,Insurance_Coverage_04M,Insurance_Coverage_05E,Insurance_Coverage_05N,Insurance_Coverage_06F,Insurance_Coverage_07P,Insurance_Coverage_08H,Insurance_Coverage_09@,Insurance_Coverage_09I,Insurance_Coverage_10A,Insurance_Coverage_12L,Insurance_Coverage_14E,Insurance_Coverage_14N,Insurance_Coverage_15F,Insurance_Coverage_15O,Insurance_Coverage_16G,Insurance_Coverage_16P,Insurance_Coverage_18@,Insurance_Coverage_18I,Insurance_Coverage_20B,Insurance_Coverage_20K,Insurance_Coverage_25G,Insurance_Coverage_26H,Insurance_Coverage_27I,Insurance_Coverage_29B,Insurance_Coverage_29K,Insurance_Coverage_32N,Insurance_Coverage_33F,Insurance_Coverage_33O,Insurance_Coverage_34P,Insurance_Coverage_35H,Insurance_Coverage_36I,Insurance_Coverage_37J,Insurance_Coverage_40M,Insurance_Coverage_41E,Insurance_Coverage_41N,Insurance_Coverage_42F,Insurance_Coverage_45@,Insurance_Coverage_46A,Insurance_Coverage_47B,Insurance_Coverage_51O,Insurance_Coverage_55J,Insurance_Coverage_56B,Insurance_Coverage_56K,Insurance_Coverage_57C,Insurance_Coverage_57L,Insurance_Coverage_65K,Insurance_Coverage_66C,Insurance_Coverage_66L,Insurance_Coverage_67D,Insurance_Coverage_68E,Insurance_Coverage_68N,Insurance_Coverage_70G,Insurance_Coverage_70P,Insurance_Coverage_71H,Insurance_Coverage_72@,fassured_1,fassured_2,fassured_3,fassured_6,fsex_0,fsex_,fsex_1,fsex_2,fmarriage_0,fmarriage_,fmarriage_1,fmarriage_2,iply_area_0286dd552c9bea9a69ecb3759e7b94777635514b,iply_area_17ba0791499db908433b80f37c5fbc89b870084b,iply_area_1d513c0bcbe33b2e7440e5e14d0b22ef95c9d673,iply_area_2d0c8af807ef45ac17cafb2973d866ba8f38caa9,iply_area_356a192b7913b04c54574d18c28d46e6395428ab,iply_area_4cd66dfabbd964f8c6c4414b07cdb45dae692e19,iply_area_4d89d294cd4ca9f2ca57dc24a53ffb3ef5303122,iply_area_511a418e72591eb7e33f703f04c3fa16df6c90bd,iply_area_59129aacfb6cebbe2c52f30ef3424209f7252e82,iply_area_6c1e671f9af5b46d9c1a52067bdf0e53685674f7,iply_area_761f22b2c1593d0bb87e0b606f990ba4974706de,iply_area_76546f9a641ede2beab506b96df1688d889e629a,iply_area_7b52009b64fd0a2a49e6d8a939753077792b0554,iply_area_92cfceb39d57d914ed8b14d0e37643de0797ae56,iply_area_98fbc42faedc02492397cb5962ea3a3ffc0a9243,iply_area_a17554a0d2b15a664c0e73900184544f19e70227,iply_area_bd307a3ec329e10a2cff8fb87480823da114f8f4,iply_area_c1dfd96eea8cc2b62785275bca38ac261256e278,iply_area_c66c65175fecc3103b3b587be9b5b230889c8628,iply_area_f1abd670358e036c31296e66b3b66c382ac00812,iply_area_fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,iply_area_fb644351560d8296fe6da332236b1f8d61b2828a,fequipment1_0,fequipment1_1,fequipment2_0,fequipment2_1,fequipment3_0,fequipment3_1,fequipment4_0,fequipment5_0,fequipment5_1,fequipment6_0,fequipment9_0,fequipment9_1,nequipment9_,nequipm

In [77]:
df_dummy_date.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988927 entries, 0 to 1988926
Columns: 218 entries, Policy_Number to Accident_Hour
dtypes: float64(16), int64(20), uint8(182)
memory usage: 986.7 MB


### 1.5 convert object (string) to numeric

In [78]:
# df[] = pd.to_numeric(df[], errors='coerce')

## 2. Define Train, Test, Y

In [79]:
#df_dummy_date.isnull().sum()

In [80]:
df = df_dummy_date 

df_train = df[df['Next_Premium']!=100]
print(df_train.shape)
df_test = df[df['Next_Premium']==100]
print(df_test.shape)
print(df_train.shape[0]+df_test.shape[0])
print(df.shape[0])

(1212659, 218)
(776268, 218)
1988927
1988927


In [81]:
df.isnull().sum().sum()

0

In [82]:
y=df_train['Next_Premium']
y.head()

0    2412.0
1    2412.0
2    3129.0
3    3129.0
4       0.0
Name: Next_Premium, dtype: float64

In [83]:
df_train=df_train.drop(['Next_Premium'], axis=1)
print(df_train.shape)

(1212659, 217)


In [84]:
df_test=df_test.drop(['Next_Premium'], axis=1)
print(df_test.shape)

(776268, 217)


### Modelling

In [85]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [86]:
train = df_train
test = df_test
y_train = y

In [ ]:
# ## downsampling for testing:
# train = df_train.head(50000) # 1,212,659
# y_train = y.head(50000)
# ## downsampling for testing:
# test=df_test.head(3000)   #776,268

In [ ]:
train.to_csv('train.csv')
test.to_csv('test.csv')
y.to_csv('y.csv')

In [ ]:
#Validation function
n_folds = 5

def mae_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    mae= np.sqrt(-cross_val_score(model, train.values, y_train.values, scoring="neg_mean_absolute_error", cv = kf))
    return(mae)

In [ ]:
# Base models

In [ ]:
# LGBM :
model_lgb = lgb.LGBMRegressor(objective='regression',n_estimators=200)
mae = mae_cv(model_lgb)
print("lgb mae: \n", mae)
print("lgb score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [ ]:
# XGBoost :
model_xgb = xgb.XGBRegressor(n_estimators=200)
mae = mae_cv(model_xgb)
print("Xgboost mae: \n", mae)
print("Xgboost score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))

In [ ]:
# RF :
model_rf=RandomForestRegressor(n_estimators=20)
mae = mae_cv(model_rf)
print("RandomForest mae: \n", mae)
print("\nRandomForest score: {:.4f} ({:.4f})\n".format(mae.mean(), mae.std()))